In [1]:
import os, sys, subprocess
conda_prefix = os.environ.get("CONDA_PREFIX", sys.prefix)  # raíz del env actual
os.environ["JAVA_HOME"] = conda_prefix
os.environ["PATH"] = f"{conda_prefix}/bin:" + os.environ["PATH"]

# Verifica que Java responde
print("JAVA_HOME:", os.environ["JAVA_HOME"])
print(subprocess.run(["java","-version"], capture_output=True, text=True).stderr)

JAVA_HOME: /opt/conda
openjdk version "11.0.27-internal" 2025-04-15
OpenJDK Runtime Environment (build 11.0.27-internal+0-adhoc..src)
OpenJDK 64-Bit Server VM (build 11.0.27-internal+0-adhoc..src, mixed mode)



In [2]:
import os
from pyspark.sql import SparkSession
from delta import configure_spark_with_delta_pip

In [3]:
base_dir = os.path.expanduser("~")  # carpeta local del cuaderno

In [4]:
builder = (
    SparkSession.builder
    .appName("spark-local-delta")
    .config("spark.sql.extensions","io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog","org.apache.spark.sql.delta.catalog.DeltaCatalog")
)
spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/opt/conda/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/sagemaker-user/.ivy2/cache
The jars for the packages stored in: /home/sagemaker-user/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4ec3086d-77f2-4b08-941d-8c3d895ee293;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.2.0 in central
	found io.delta#delta-storage;3.2.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
downloading https://repo1.maven.org/maven2/io/delta/delta-spark_2.12/3.2.0/delta-spark_2.12-3.2.0.jar ...
	[SUCCESSFUL ] io.delta#delta-spark_2.12;3.2.0!delta-spark_2.12.jar (294ms)
downloading https://repo1.maven.org/maven2/io/delta/delta-storage/3.2.0/delta-storage-3.2.0.jar ...
	[SUCCESSFUL ] io.delta#delta-storage;3.2.0!delta-storage.jar (152ms)
downloading https://repo1.maven.org/maven2/org/antlr/antlr4-runtime/4.9.3/antlr4-runtime-4.9.3.jar ...
	[SUCCESSFUL ] org.antlr#antlr4-runtime;4.9.3!antlr4-runtime.jar (87ms)
:: resolution report :: re

In [48]:
# Lee el CSV local que hayas subido al Jupyter (sin S3)
csv_path   = f"file://{base_dir}/STAR.csv"
delta_path = f"file://{base_dir}/delta/STAR"
parquet_path = f"file://{base_dir}/parquet/STAR"


In [52]:
df = (spark.read.format("csv").option("header", True).option("inferSchema", True).load(csv_path))

In [53]:
(df.write.format("delta").mode("overwrite").save(delta_path))

In [54]:
spark.read.format("delta").load(delta_path).show(5)

[Stage 267:==================================================>    (46 + 2) / 50]

+----+------+---------+-------+------------+-----+------------+------------+-----+-----+-----+-----+-----+-----+-----+-----+--------+------+--------+--------+----------+--------+--------+--------+--------+--------+--------+--------+---------+---------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-------+-------+-------+-------+---------+---------+---------+---------+
|  c0|gender|ethnicity|  birth|       stark|star1|       star2|       star3|readk|read1|read2|read3|mathk|math1|math2|math3|  lunchk|lunch1|  lunch2|  lunch3|   schoolk| school1| school2| school3| degreek| degree1| degree2| degree3|  ladderk|  ladder1|   ladder2|   ladder3|experiencek|experience1|experience2|experience3|tethnicityk|tethnicity1|tethnicity2|tethnicity3|systemk|system1|system2|system3|schoolidk|schoolid1|schoolid2|schoolid3|
+----+------+---------+-------+------------+-----+------------+------------+-----+-----+-----+-----+-----+-----+--